In [1]:
import copy
import numpy as np
from os.path import isfile
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
from torchvision import models
from torchvision import transforms

from image_loader import ProductImageCategoryDataset, create_data_loaders
from neural_network_models import ImageModel, train

In [2]:
%reload_ext tensorboard
%tensorboard --logdir runs

In [3]:
# Checks if GPU is avaliable to run on.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# Other batch hyperparameters
validation_split = 0.2
batch_size = 8

# Create data loaders
data_loader, dataset_sizes = create_data_loaders("data/cleaned_images_128/", ProductImageCategoryDataset, 128, data_transforms, validation_split, batch_size)

# Create model class
model = ImageModel().to(device)
optimiser = torch.optim.SGD(model.parameters(), lr = 0.001, momentum=0.9) 
scheduler = lr_scheduler.StepLR(optimiser, step_size = 10, gamma=0.1)

In [5]:
model = train(40, model, optimiser,data_loader, dataset_sizes, scheduler=scheduler, model_save_location="/content/drive/My Drive/coding/vision_model_state.pt", device=device)

---------------
Epoch number: 0
---------------
Training...
